In [ ]:
!pip uninstall -y tensorboard
!pip uninstall -y tensorflow
!pip uninstall -y autoai-libs

!pip install -U geocoder
!pip install -U geopy
!pip install -U googlemaps
!pip install -U folium
!pip install -U numpy
!pip install -U pandas

import pandas as pd
import numpy as np

from pandas import json_normalize  # tranform JSON file into a pandas dataframe

import googlemaps
from datetime import datetime
import requests
import geopy

geopy.geocoders.options.default_user_agent = "my-application"

import ast
import json
import time

import folium
from folium.plugins import HeatMap

from ibm_botocore.client import Config
import ibm_boto3

print('Ready')

In [2]:
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
credentials = {
    'IAM_SERVICE_ID': 'iam-ServiceId-c718bc9c-6715-41c4-a090-ef49d30cf02e',
    'IBM_API_KEY_ID': 'nEAwnadbTDR7sZjg2dsuEPEqMP3WU1UFgh7WLkkMZppS',
    'ENDPOINT': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.eu-gb.bluemix.net/oidc/token',
    'BUCKET': 'capstoneprojectthebattleofneighbo-donotdelete-pr-rpev4qjyoaz4ga'
}

In [3]:
def download_file_cos(credentials, local_file_name, key):
    cos = ibm_boto3.client(service_name='s3',
                           ibm_api_key_id=credentials['IBM_API_KEY_ID'],
                           ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
                           ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
                           config=Config(signature_version='oauth'),
                           endpoint_url=credentials['ENDPOINT'])
    try:
        res = cos.download_file(Bucket=credentials['BUCKET'], Key=key, Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')

In [4]:
def upload_file_cos(credentials, local_file_name, key):
    cos = ibm_boto3.client(service_name='s3',
                           ibm_api_key_id=credentials['IBM_API_KEY_ID'],
                           ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
                           ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
                           config=Config(signature_version='oauth'),
                           endpoint_url=credentials['ENDPOINT'])
    try:
        res = cos.upload_file(Filename=local_file_name, Bucket=credentials['BUCKET'], Key=key)
    except Exception as e:
        print(Exception, e)
    else:
        print(' File Uploaded')

In [5]:
download_file_cos(credentials, 'hotels_all.csv', 'hotels_all.csv')
download_file_cos(credentials, 'food_Hotels.csv', 'food_Hotels.csv')
download_file_cos(credentials, 'location_geometry.csv', 'location_geometry.csv')

File Downloaded
File Downloaded
File Downloaded


In [6]:
df_all_hotels_sorted = pd.read_csv('hotels_all.csv', sep=',', encoding='utf_8', index_col=False).reset_index()
df_all_hotels_sorted.shape

(785, 8)

In [7]:
df_all_hotels_sorted.head()

,index,id,location,name,food.count,nightlife.count,food.ratio,nightlife.ratio
0,0,5b2d6e07b2958f002c54e4ba,"{'lat': 37.090429, 'lng': -8.209487, 'labeledL...",1-211 appartement,30,17,0.60,0.34
1,1,4f7350c2e4b0257534d4008b,"{'address': 'Algarve', 'lat': 37.10418, 'lng':...",402,9,2,0.18,0.04
2,2,575882b6498e96a54155636e,"{'lat': 37.093356, 'lng': -8.743884, 'labeledL...",A Cabana B&B,10,2,0.20,0.04
3,3,5011b9abe4b0c3be96766adf,"{'lat': 37.09199419481045, 'lng': -8.209547670...",Acqua Maris,33,17,0.66,0.34
4,4,4da4481b2939b1f7049e2e57,"{'address': 'Rocha Baixinha', 'lat': 37.082724...",Adriana Beach Club & Spa,9,5,0.18,0.10


In [8]:
df_food_Hotels_sorted = pd.read_csv('food_Hotels.csv', sep=',', encoding='utf_8', index_col=False).reset_index()
df_food_Hotels_sorted.shape

(3691, 4)

In [9]:
df_location = pd.read_csv('location_geometry.csv', sep=',', encoding='utf_8', index_col=False).reset_index()[['geometry.location.lat', 'geometry.location.lng']]
df_location.head()

,geometry.location.lat,geometry.location.lng
0,37.089072,-8.247880
1,37.130970,-8.348275
2,37.469978,-7.473078
3,37.334386,-8.489073
4,37.163989,-8.304248


In [10]:
map_center_lat = df_location['geometry.location.lat'].mean()
map_center_lng = df_location['geometry.location.lng'].mean()
print(map_center_lat)
print(map_center_lng)

37.18812233287672
-8.179172341095889


In [11]:
all_hotels2show_map = pd.DataFrame(df_all_hotels_sorted)

for ind in df_all_hotels_sorted.index:
    location = df_all_hotels_sorted['location'][ind]
    location_json = ast.literal_eval(location)
    try:
        coord = pd.DataFrame(location_json['labeledLatLngs'])
        latitude = coord['lat'][0]
        longitude = coord['lng'][0]
    except KeyError:
        latitude = location_json['lat']
        longitude = location_json['lng']

    all_hotels2show_map.loc[ind, 'latitude'] = latitude
    all_hotels2show_map.loc[ind, 'longitude'] = longitude

all_hotels2show_map_food_ratio = all_hotels2show_map[['latitude', 'longitude', 'food.ratio']]
all_hotels2show_map_nightlife_ratio = all_hotels2show_map[['latitude', 'longitude', 'nightlife.ratio']]
print(all_hotels2show_map.shape)

(785, 10)


In [12]:
all_hotels2show_map_food_ratio.head()

,latitude,longitude,food.ratio
0,37.090429,-8.209487,0.60
1,37.104180,-8.116300,0.18
2,37.093356,-8.743884,0.20
3,37.091994,-8.209548,0.66
4,37.082725,-8.147276,0.18


In [13]:
all_hotels2show_map_nightlife_ratio.head()

,latitude,longitude,nightlife.ratio
0,37.090429,-8.209487,0.34
1,37.104180,-8.116300,0.04
2,37.093356,-8.743884,0.04
3,37.091994,-8.209548,0.34
4,37.082725,-8.147276,0.10


In [14]:
# create map using latitude and longitude values
map_Algarve = folium.Map(location=[map_center_lat, map_center_lng], zoom_start=10)

for ind in df_all_hotels_sorted.index:
    venue = df_all_hotels_sorted['name'][ind]

    location = df_all_hotels_sorted['location'][ind]
    try:
        location_json = ast.literal_eval(location)
        coord = pd.DataFrame(location_json['labeledLatLngs'])
        latitude = coord['lat'][0]
        longitude = coord['lng'][0]
    except KeyError:  # Some of the locations does not have 'labeledLatLngs', but just JSON string
        try:
            location_json = ast.literal_eval(location)
            latitude = location_json['lat']
            longitude = location_json['lng']
        except ValueError:  # Some of those JSON string does not have 'quotes' around the values
            latitude = location['lat']
            longitude = location['lng']

    name = u'Hotel: {}'.format(venue)
    label = folium.Popup(name, parse_html=True, max_width=2650)

    folium.CircleMarker(
        [latitude, longitude],
        radius=2,
        popup=label,
        color='black',
        fill=True,
        # fill_color='#3186cc',
        fillOpacity=0.7).add_to(map_Algarve)

HeatMap(all_hotels2show_map_food_ratio,gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, min_opacity=0.5, max_opacity=0.8).add_to(map_Algarve)
map_Algarve

In [15]:
# create map using latitude and longitude values
map_Algarve = folium.Map(location=[map_center_lat, map_center_lng], zoom_start=10)

for ind in df_all_hotels_sorted.index:
    venue = df_all_hotels_sorted['name'][ind]

    location = df_all_hotels_sorted['location'][ind]
    try:
        location_json = ast.literal_eval(location)
        coord = pd.DataFrame(location_json['labeledLatLngs'])
        latitude = coord['lat'][0]
        longitude = coord['lng'][0]
    except KeyError:  # Some of the locations does not have 'labeledLatLngs', but just JSON string
        try:
            location_json = ast.literal_eval(location)
            latitude = location_json['lat']
            longitude = location_json['lng']
        except ValueError:  # Some of those JSON string does not have 'quotes' around the values
            latitude = location['lat']
            longitude = location['lng']

    name = u'Hotel: {}'.format(venue)
    label = folium.Popup(name, parse_html=True, max_width=2650)

    folium.CircleMarker(
        [latitude, longitude],
        radius=2,
        popup=label,
        color='black',
        fill=True,
        # fill_color='#3186cc',
        fillOpacity=0.7).add_to(map_Algarve)

HeatMap(all_hotels2show_map_nightlife_ratio,gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, min_opacity=0.5, max_opacity=0.8).add_to(map_Algarve)
map_Algarve